# Lambda Calculus PyCon 2019

We only have single argument functions.  
  
No packages/modules  
No objects  
No numbers  
No strings  
No datatypes  
.  
.  
.  

In [1]:
def f(x):
    return x


def f(x):
    return x(x)


def f(x):
    def g(y):
        return x(y)

    return g

## Modelling a switch

In [2]:
def LEFT(a):
    def f(b):
        return a

    return f


def RIGHT(a):
    def f(b):
        return b

    return f

In [3]:
LEFT('5v')('gnd')

'5v'

In [4]:
RIGHT('5v')('gnd')

'gnd'

In [5]:
def add(x, y):
    return x + y


add(2, 3)

5

#### Currying

Take a *multiple argument* function and turning it into a *single* argument function.

In [6]:
def add(x):
    def f(y):
        return x + y

    return f


add(2)(3)

5

In [7]:
def TRUE(x):
    return lambda y: x


def FALSE(x):
    return lambda y: y

In [8]:
TRUE('5v')('gnd')

'5v'

In [9]:
FALSE('5v')('gnd')

'gnd'

## Boolean operators

### NOT

In [10]:
def NOT(x):
    return x(FALSE)(TRUE)


assert NOT(TRUE) is FALSE
assert NOT(FALSE) is TRUE

In [11]:
print(NOT(TRUE))

<function FALSE at 0x10b7899d8>


In [12]:
print(NOT(FALSE))

<function TRUE at 0x10b789950>


### AND

In [13]:
print(2 and 3)
print(0 and 3)

3
0


**Behaviour:** if the first value is *True* then Python looks at the second value to check result. If the first value is *False* the it automatically returns *False*.

If $x$ is *True*, return $y$. If $x$ is *False* return *False*

In [14]:
def AND(x):
    return lambda y: x(y)(x)

In [15]:
print(AND(TRUE)(TRUE))
print(AND(FALSE)(TRUE))
print(AND(TRUE)(FALSE))
print(AND(FALSE)(FALSE))

<function TRUE at 0x10b789950>
<function FALSE at 0x10b7899d8>
<function FALSE at 0x10b7899d8>
<function FALSE at 0x10b7899d8>


### OR

In [16]:
print(2 or 3)
print(0 or 3)

2
3


**Behaviour:** if the first value is *True* then return the first value (True). If the first value is *False* then return second value.

If (x) is *True*, return *True*. If (x) is *False* return *y*.

In [17]:
def OR(x):
    return lambda y: x(x)(y)

In [18]:
print(OR(TRUE)(TRUE))
print(OR(FALSE)(TRUE))
print(OR(TRUE)(FALSE))
print(OR(FALSE)(FALSE))

<function TRUE at 0x10b789950>
<function TRUE at 0x10b789950>
<function TRUE at 0x10b789950>
<function FALSE at 0x10b7899d8>


## Numbers

There wont be *real* numbers, just abstractions.

In [19]:
ONE = lambda f: lambda x: f(x)
TWO = lambda f: lambda x: f(f(x))
THREE = lambda f: lambda x: f(f(f(x)))
FOUR = lambda f: lambda x: f(f(f(f(x))))

In [20]:
# NOT allowed
def incr(x):
    return x + 1


def p(t):
    return (t[0] + 1, t[0])


print(THREE(incr)(0))
print(THREE(p)((0, 0)))

3
(3, 2)


In [21]:
a = FOUR(THREE)
a(incr)(0)

81

(It is doing exponentiation)

In [22]:
ZERO = lambda f: lambda x: x

ZERO(incr)(0)

0

## Arithmetic

Implement successor (*counting*)

### SUCCESSOR

`n(f)(x)` is the old number  
`(lambda f: lambda x: f(n(f)(x)))` = add one more `f(x)` like we were doing before

In [23]:
SUCC = lambda n: (lambda f: lambda x: f(n(f)(x)))
SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

In [24]:
SUCC(FOUR)

<function __main__.<lambda>.<locals>.<lambda>(f)>

In [25]:
a = SUCC(FOUR)


a(incr)(0)

5

### ADDITION

In [26]:
ADD = lambda x: lambda y: y(SUCC)(x)

In [27]:
a = ADD(FOUR)(THREE)

In [28]:
a(incr)(0)

7

$x$ repetitions of **f()**

### MULTIPLICATION

`y(x(f))` = `x` repetitions of `f()`, `y` times.

In [29]:
MUL = lambda x: lambda y: lambda f: y(x(f))

In [30]:
m = MUL(FOUR)(THREE)

In [31]:
m(incr)(0)

12

In [32]:
# TWO could be considered like going down in a dictionary

data = {
    'a': {
        'b': {
            'c': 42
        }
    }
}


def getc(d):
    return d['a']['b']['c']

getc(data)

42

But if we get malformed data we get an error.

In [33]:
getc({})

KeyError: 'a'

To fix it we may need to write a lot of repetitive code to check for input 'correctness':

In [34]:
def getc(d):
    d = d.get("a")
    if d is not None:
        d = d.get("b")
    if d is not None:
        d = d.get("c")
    return d


getc(data)

42

And now if we get an empty dictionary we don't get an error.

In [35]:
getc({})

In [36]:
def perhaps(d, func):
    if d is not None:
        return func(d)
    else:
        return None

In [37]:
perhaps(data, lambda d: d.get('a'))

{'b': {'c': 42}}

In [38]:
perhaps({}, lambda d: d.get('a'))

In [39]:
perhaps(perhaps(data, lambda d: d.get("a")),\
        lambda d: d.get("b"))

{'c': 42}

In [40]:
perhaps(perhaps(perhaps(data, lambda d: d.get("a")),\
        lambda d: d.get("b")), lambda d: d.get('c'))

42

Monad

![](substitution.png)

## Data structures

In [41]:
# Lisp
# (cons 2 3)   -> (2, 3)
# (car p)      -> (2)
# (cdr p)      -> (3)

In [42]:
def cons(a, b):
    def select(m):
        if m == 0:
            return a
        elif m == 1:
            return b

    return select

In [43]:
p = cons(2, 3)
p

<function __main__.cons.<locals>.select(m)>

In [44]:
print(p(0))
print(p(1))

2
3


In [45]:
CONS = lambda a: lambda b: (lambda s: s(a)(b))

In [46]:
p = CONS(2)(3)
p

<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>(s)>

Using the switch we built at the beginning:

In [47]:
p(TRUE)

2

In [48]:
p(FALSE)

3

In [49]:
p = CONS(2)(CONS(3)(4))
p

<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>(s)>

In [54]:
print(p(TRUE))
print(p(FALSE)(FALSE))
print(p(FALSE)(TRUE))

2
4
3


In [64]:
CAR = lambda p: p(TRUE)
CDR = lambda p: p(FALSE)

## SUBSTRACTION

Starti with the pair `(0, 0)` and use de **successor** until you reach the number.

In [76]:
def t(p):
    return (p[0] + 1, p[0])

THREE(p)((0, 0))

(3, 2)

`t[0]` is `CAR(t)` 

`t[0] + 1` is the `SUCC` of `CAR(t)``


In [77]:
T = lambda p: CONS(SUCC(CAR(p)))(CAR(p))

In [78]:
a = FOUR(T)(CONS(ZERO)(ZERO))

In [79]:
CAR(a)(incr)(0)

4

In [80]:
CDR(a)(incr)(0)

3

### PREDECESSOR

Take a number `n` and apply that number to the `T` function we declared before, so we are going our way up with the tuple. And the we take the `CDR`.

In [87]:
PRED = lambda n: CDR(n(T)(CONS(ZERO)(ZERO)))

In [88]:
PRED(FOUR)(incr)(0)

3

### SUBSTRACTION

In [89]:
SUB = lambda x: lambda y: y(PRED)(x)

In [93]:
a = SUB(FOUR)(TWO)

In [94]:
a(incr)(0)

2

### EQUALITY

`lambda f: FALSE` we disregard the argument and always return `FALSE`

In [96]:
ISZERO = lambda n: n(lambda f: FALSE)(TRUE)

In [99]:
print(ISZERO(ZERO))
print(ISZERO(TWO))

<function TRUE at 0x10b789950>
<function FALSE at 0x10b7899d8>


## Recurssion

### FACTORIAL

In [101]:
# 'regular' python
def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n - 1)


fact(4)

24

**NOTES** for the following code:

`ISZERO()` is going to return `TRUE` or `FALSE`, which act like and *if statement*.

In [103]:
FACT = lambda n: ISZERO(n)(ONE)(MUL(n(FACT(PRED(n)))))